# 1、使用FUNCTION_CALL模式

In [ ]:
from langchain.memory import ConversationBufferMemory
import hub
# 1.导入相关包
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
# 假设 llm 的定义和 os.environ 的设置在前面已完成，这里仅添加可能需要的导入
from langchain_openai import ChatOpenAI
import os
import dotenv


dotenv.load_dotenv()
# 2.定义搜索工具
# ① 设置 TAVILY_API 密钥
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY") # 需要替换为你的 Tavily API 密钥

# ② 定义搜索工具
search = TavilySearchResults(max_results=1)

# 3.自定义提示词模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "您是一位乐于助人的助手，请务必使用 tavily_search_results_json 工具来获取信息。"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# 注意: agent_scratchpad 必须声明，它用于存储和传递 Agent 的思考过程。比如，在调用链式工具时（如搜索天气再推荐行程），agent_scratchpad 保留所有历史步骤，避免上下文丢失。format 方法会将 intermediate_steps 转换为特定格式的字符串，并赋值给 agent_scratchpad 变量。如果不传递 intermediate_steps 参数，会导致 KeyError: 'intermediate_steps' 错误。

# 提供记忆的实例: ConversationBufferMemory
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history", # 此值需要与提示词模板中的记录记忆的变量同名
)

# 4.定义LLM
llm = ChatOpenAI(
    model="gpt-5-mini",
    # temperature=0,
)

# 5.创建Agent对象
agent = create_tool_calling_agent(
    llm = llm,
    tools = [search],
    prompt = prompt
)

# 6.创建AgentExecutor执行器
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search],
    verbose=True,
    memory = memory,
    max_iterations=5 # 限制最大agent递归次数，防止无线循环
)

# 7.测试
agent_executor.invoke({"input": "今天北京的天气怎么样？"})

In [ ]:
result = agent_executor.invoke({"input": "上海呢？"})

print(result)

# 2、使用ReAct模式

In [ ]:
# 1.导入相关包
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI # 补充导入 ChatOpenAI
from langchain.memory import ConversationBufferMemory # 补充导入 ConversationBufferMemory
import os

# 2.定义搜索工具
# ① 设置 TAVILY_API 密钥
os.environ["TAVILY_API_KEY"] = "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" # 需要替换为你的 Tavily API 密钥

# ② 定义搜索工具
search = TavilySearchResults(max_results=1)

# ③ 设置工具集
tools = [search]

# 3.自定义提示词模板
template = (
    "Assistant is a large language model trained by OpenAI.\n\n"
    "Assistant is designed to be able to assist with a wide range of tasks, from answering "
    "simple questions to providing in-depth explanations and discussions on a wide range of topics.\n\n"
    "As a language model, Assistant is able to generate human-like text based on the input it "
    "receives, allowing it to engage in natural-sounding conversations and provide responses that "
    "are coherent and relevant to the topic at hand.\n\n"
    "Assistant is constantly learning and improving, and its capabilities are constantly "
    "evolving. It is able to process and understand large amounts of text, and can use this "
    "knowledge to provide accurate and informative responses to a wide range of questions. "
    "Additionally, Assistant is able to generate its own text based on the input it receives, allowing it "
    "to engage in discussions and provide explanations and descriptions on a wide range of "
    "topics.\n\n"
    "Overall, Assistant is a powerful tool that can help with a wide range of tasks and "
    "provide valuable insights and information on a wide range of topics. Whether you need help "
    "with a specific question or just want to have a conversation about a particular topic, Assistant is "
    "here to assist.\n\n"

    "TOOLS:\n"
    "-----\n"
    "Assistant has access to the following tools:\n\n"
    "{tools}\n\n"
    "To use a tool, please use the following format:\n\n"
    "Thought: Do I need to use a tool? Yes\n"
    "Action: the action to take, should be one of [{tool_names}]\n"
    "Action Input: the input to the action\n"
    "Observation: the result of the action\n"
    "...\n"
    "When you have a response to say to the Human, or if you do not need to use a tool, "
    "you MUST use the format:\n\n"
    "Thought: Do I need to use a tool? No\n"
    "Final Answer: [your response here]\n\n"

    "Begin!\n\n"

    "# 提示: 助手是由 OpenAI 训练的一个大型语言模型。\n"
    "# 助手旨在能够协助处理各种各样的任务, 从回答简单的问题到对各种主题进行深入的解释和讨论。作为语言模型, 助手能够根据收到的输入生成类似人类的文本, 从而能够进行自然流畅的对话, 并提供与当前主题相关的连贯的回答。\n"
    "# 助手在不断学习和改进, 其能力也在不断发展。它能够处理和理解大量的文本, 并利用这些知识为各种问题提供准确且有信息量的回答。此外, 助手还能根据收到的输入生成自己的文本, 从而能够参与讨论, 并对各种主题提供解释和描述。\n"
    "# 总的来说, 助手是一个强大的工具, 能够帮助完成各种各样的任务, 并在各种主题上提供有价值的见解和信息。无论您是需要解答某个具体问题, 还是只是想就某个特定话题进行交流, 助手都在这里为您提供帮助。\n"
    "# 工具:\n"
    "# ---\n"
    "# 助手可以使用以下工具: \n"
    "{tools}\n"
    "# 若要使用某个工具, 请使用以下格式:\n"
    "# ---\n"
    "# 思考: 我是否需要使用工具? 是\n"
    "# 行动: 要采取的行动, 应为 [{tool_names}] 中的一个\n"
    "# 行动输入: 行动的输入\n"
    "# 观察: 行动的结果\n"
    "# ...\n"
    "# 当您有回复要对人类说, 或者不需要使用工具时, 您必须使用以下格式: \n"
    "# ---\n"
    "# 思考: 我是否需要使用工具? 否\n"
    "# 最终答案: [您的回复在此]\n\n"

    "# 开始! \n\n"
    "# 之前的对话历史:\n"
    "{chat_history}\n\n"

    "# 新的输入: {input}\n"
    "{agent_scratchpad}\n"
    "Thought:"
)
prompt = PromptTemplate.from_template(template)

# 4.定义LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

# 5.定义记忆组件(以ConversationBufferMemory为例)
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# 6.创建Agent对象
agent = create_react_agent(llm, tools, prompt)

# 7.创建AgentExecutor执行器
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, memory=memory)

# 8.测试
agent_executor.invoke({"input": "我的名字叫Bob"})

In [ ]:
agent_executor.invoke({"input": "我的名字叫什么?"})
